<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/nlp-for-vector-similarity-search/02_nearest_neighbor_indexes_for_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Nearest Neighbor Indexes for Similarity Search

**Reference**

[Nearest Neighbor Indexes for Similarity Search](https://www.pinecone.io/learn/vector-indexes/)

##Setup

In [ ]:
!pip install faiss-cpu
!pip install -U sentence-transformers

In [ ]:
import shutil
import urllib.request as request
from contextlib import closing
from io import StringIO
import pandas as pd
import numpy as np
import tarfile

from sentence_transformers import SentenceTransformer

import faiss

##Dataset

In [ ]:
# first we download the Sift1M dataset
with closing(request.urlopen("ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz")) as res:
  with open("sift.tar.gz", "wb") as f:
    shutil.copyfileobj(res, f)

In [ ]:
# download leaves us with a tar.gz file, we unzip it
tar = tarfile.open("sift.tar.gz", "r:gz")
tar.extractall()

In [ ]:
# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
  a = np.fromfile(fp, "int32")
  d = a[0]
  return a.reshape(-1, d + 1)[:, 1:].copy().view("float32")

In [ ]:
#  data we will search through
xb = read_fvecs("./sift/sift_base.fvecs")  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [ ]:
xq.shape

(1, 128)

In [ ]:
xb.shape

(1000000, 128)

In [ ]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

##Flat Index

In [ ]:
# dimensionality of Sift1M data
d = 128
# number of nearest neighbors to return
k = 10

In [ ]:
# initialize IndexFlatL2 index
index = faiss.IndexFlatIP(d)
index.add(xb)

In [ ]:
%%time

# search
D, I = index.search(xq, k)
print(I)

[[932085 934876 561813 708177 706771 695756 435345 701258 872728 455537]]
CPU times: user 33.7 ms, sys: 136 µs, total: 33.8 ms
Wall time: 34.1 ms


In [ ]:
# [f'{i}: {sentences[i]}' for i in I[0]]

## Locality Sensitive Hashing

In [ ]:
# resolution of bucketed vectors
nbits = d * 4

# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.add(xb)

# search
D, I = index.search(xq, k)

In [ ]:
print(I)

[[435345 931632 708177 813701 934876 455537 932085 561813 248185 361496]]


##HNSW Implementation

In [ ]:
# set HNSW index parameters
M = 64                # number of connections each vertex will have
ef_search = 32        # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search

# add data to index
index.add(xb)

# search as usual
D, I = index.search(xq, k)
print(I)

[[932085 934876 561813 708177 706771 695756 435345 701258 455537 872728]]


##Inverted File Index